In [ ]:
import numpy as np
from PIL import ImageFile
import pandas as pd
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.backend import *
from tensorflow.keras.preprocessing.image import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.applications import *
from tensorflow.keras.models import *
import tensorflow as tf 
ImageFile.LOAD_TRUNCATED_IMAGES = True
plt.style.use('fivethirtyeight')
%matplotlib inline
import cv2
import sklearn
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product

In [ ]:
df=pd.read_csv('../input/aiijc-final-dcm/AIJ_2gis_data/train.csv')

In [ ]:
X_train, _, Y_train, _ = train_test_split(df['filename'],
                                          df['label'], 
                                          test_size=0.0000000000001,
                                          shuffle = True, 
                                          random_state = 60)

In [ ]:
df_new =pd.DataFrame(data={'filename':X_train, 
                           'label':Y_train})
df_new.reset_index(drop=True, inplace=True)

In [ ]:
test_aug = ImageDataGenerator()

In [ ]:
labels_list = ['3_24+10', '3_24+30', '3_24+5', '3_24+60', '3_24+70', '3_24+90', '3_24+110', '3_24+20', '3_24+40', '3_24+50', '3_24+80', '1_12_1', '1_12_2', '1_13', '1_14', '1_19', '1_24', '1_26', '1_27', '3_21', '3_31', '3_33', '4_4_1', '4_4_2', '4_5_2', '4_5_4', '4_5_5', '4_8_5', '4_8_6', '5_17', '6_2+50', '6_2+70', '6_2+30', '6_2+40', '6_2+60', '6_2+80', '6_7', '7_1', '7_11', '7_13', '7_14', '7_2', '7_4', '7_7', '7_9', 'smoke', 'unknown', '1_11_1', '1_11_2', '1_15', '1_16', '1_18', '1_20_1', '1_22', '1_25', '1_28', '1_29', '1_30', '1_8', '2_3_1', '2_3_L', '2_3_R', '2_6', '2_7', '3_15', '3_17', '3_20', '3_25+70', '3_25+20', '3_25+30', '3_25+40', '3_25+50', '3_25+5', '3_25+60', '3_6', '4_1_6', '4_2_1', '4_2_2', '5_15_5', '6_3_1', '7_3', '7_6', '1_17', '3_16', '5_15_3', '5_20', '7_12', '1_31', '3_10', '3_19', '3_2', '3_5', '3_7', '3_9', '4_1_2_1', '4_1_3_1', '4_5_1', '4_5_6', '4_8_1', '4_8_2', '4_8_3', '5_1', '5_11_1', '5_12_1', '5_13_1', '5_13_2', '5_14_1', '5_14_2', '5_14_3', '5_2', '5_23_2', '5_24_2', '5_3', '5_4', '5_8', '7_5', '3_32', '7_18', '1_2', '1_33', '1_7', '2_4', '3_18_1', '3_18_2', '3_8', '4_1_2', '4_1_3', '5_14', '6_15_2', '6_15_3', '6_6', '6_8_1', '1_1', '1_20_2', '1_20_3', '1_21', '1_23', '1_5', '2_1', '2_2', '2_5', '3_1', '3_26', '3_27', '3_28', '3_29', '3_30', '4_1_1', '4_1_4', '4_1_5', '4_2_3', '4_3', '4_8_4', '5_16', '5_18', '5_19', '5_21', '5_22', '5_5', '5_6', '5_7_1', '5_7_2', '5_9', '6_15_1', '6_16', '6_4', '6_8_2', '6_8_3', '5_29', '5_31+10', '5_31+20', '5_31+30', '5_31+40', '5_31+5', '5_31+50', '5_32', '5_33', '1_6', '5_15_2+2', '5_15_2+1', '5_15_2+3', '5_15_2+5']

In [ ]:
images = []
root_dir = "../input/d/congthjnh/aaaaaaaaaa/img/img/"
labels = []
X = []

for i in tqdm(range(0, len(df_new['filename']))):
  img_path = root_dir + df_new["filename"][i][13:]
  X.append(cv2.resize(cv2.imread(img_path),dsize=(80,80))/255.)
  labels.append(labels_list.index(df_new["label"][i]))
    
X = np.array(X)
y = np.array(labels)

In [ ]:
clear_session()
model_1= load_model('../input/d/congthjnh/aaaaaaaaaa/VGG19_2.h5')
y_pred_1 = model_1.predict(test_aug.flow(X, batch_size = 512, shuffle=False), verbose=1)
model_2 = load_model('../input/d/congthjnh/aaaaaaaaaa/InceptionResNetV2_2.h5')
y_pred_2 = model_2.predict(test_aug.flow(X, batch_size = 512, shuffle=False), verbose=1)
model_3=load_model('../input/d/congthjnh/aaaaaaaaaa/denset201_2.h5')
y_pred_3 = model_3.predict(test_aug.flow(X, batch_size = 512, shuffle=False), verbose=1)

In [ ]:
def ensemble_predictions(weights=None):
    yhats = [y_pred_1,y_pred_2, y_pred_3]
    yhats = array(yhats)
    summed = tensordot(yhats, weights, axes=((0),(0)))
    result = argmax(summed, axis=1)
    del yhats
    del summed
    return result

In [ ]:
def evaluate_ensemble(weights=None, testy=y):
    yhat = ensemble_predictions(weights)
    a= sklearn.metrics.f1_score(testy,yhat, average='micro').astype('float64')
    del yhat
    return a

In [ ]:
def normalize(weights):
    result = norm(weights, 1)
    if result == 0.0:
        return weights
    final= weights / result
    del weights
    del result
    return final

In [ ]:
def grid_search(members=['a', 'b','c']):
    w = list(np.arange(0.0,1.0,0.1))
    best_score, best_weights = 0, 1/len(members)
    for weights in product(w, repeat=len(members)):
        if len(set(weights)) == 1:
            continue
        weights = normalize(weights)
        score = evaluate_ensemble(weights=weights)
        if score > best_score:
            best_score, best_weights = score, weights
            print((best_weights, best_score))
    return list(best_weights)

In [ ]:
weights_for_ensemble = grid_search()